In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

#https://habrahabr.ru/company/ods/blog/327242/

import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
%matplotlib inline
import seaborn as sb
from sklearn.preprocessing import StandardScaler


import matplotlib.cm as cm
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import __version__
from plotly import graph_objs as go
from plotly.graph_objs import *


from pylab import rcParams
rcParams['figure.figsize'] = 35, 40

## Читаем данные

In [ ]:
df=pd.read_pickle('MG_Sales_customer.pickle',compression='gzip')

#выбираем покупателей для дальнейшего анализа
sales_sum=df.groupby('Покупатель')['Количество'].sum()
#выкидываем со слишком большими продажами (сводные карты) и тех кто купил один раз
sales_sum.drop(sales_sum[(sales_sum>133)|(sales_sum==1)].index, inplace=True)
customers_name=list(sales_sum.index)
del sales_sum

#делаем выборку
select=df.loc[df['Покупатель'].isin(customers_name),['Покупатель','ВидИзделия','ПодвидИзделия','СтильДизайна','ВидДизайна','ПодвидДизайна','ОсновнойКамень','ГруппаТовара','Коллекция','МаркетинговаяЛинейка','ЦветМеталла','ЦветПокрытия','ТоварСреднийВес','Тип','Размер','Вес']]
#select['ПокупательДатаРождения']=select['ПокупательДатаРождения'].fillna(dt.datetime(999,1,1))
del customers_name

#шкалируем
categorical_columns = [c for c in select.columns if select[c].dtype.name == 'object']
select_fact=select.copy()
for col in categorical_columns:
    select_fact[col]=pd.factorize(select_fact[col])[0]
    
scaler = StandardScaler()
X_scaled = pd.DataFrame(data=scaler.fit_transform(select_fact), columns=select_fact.columns)
#del select
del select_fact
del df


db = DBSCAN(eps=0.2,n_jobs=-1).fit(X_scaled)

In [ ]:
init_notebook_mode(connected = True)

val='ВидИзделия'
ЦветМеталла=list(select[val].unique())
colors = list(iter(cm.rainbow(np.linspace(0, 1, len(ЦветМеталла)))))
cl=select.loc[:30000,val].map(lambda xx: ЦветМеталла.index(xx))

trace1 = go.Scatter3d(#
    x=tsne_representation[:, 0],
    y=tsne_representation[:, 1],
    z=tsne_representation[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        colorscale='Rainbow',#Greys, YlGnBu, Greens, YlOrRd, Bluered, RdBu, Reds, Blues, Picnic, Rainbow, Portland, Jet, Hot, Blackbody, Earth, Electric, Viridis
        opacity=0.8,
        color=cl,
        colorbar=ColorBar(
                title=val
        )
    )
)

data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='T-SNE продаж')
print(ЦветМеталла)